In [1]:
import pymysql
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import arango
import pyArango.connection as ADB

# 1. Deal with User collection

In [2]:
# connect to MySQL database through AWS RDS
db = pymysql.connect(host="yelp.ciyx8eht0ylz.us-east-1.rds.amazonaws.com", 
                     port=3306, 
                     user="melody", 
                     password="19931206Wr.", 
                     database="yelp", 
                     charset='utf8')
cursor = db.cursor()

In [8]:
# extract user collection from MySQL
user = pd.read_sql("""select  *
                        from yelp.user
                        ;""",
                 db)
user.head(10)

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends
0,---2PmXbF47D870stH1jqA,Susan,35,2012-10-24 13:20:46,63,4,36,,"s7cUp9ma9h9FYN-fa0FtgA, o9QkuHIyxmqbORF7PeAbuw..."
1,---7FS-3SMy-cnnIPHcW9w,Nick,1,2014-12-28 16:02:17,0,0,0,,None
2,--0DrQkM0FT-yCQRWw82uQ,Jimmy,25,2007-11-13 15:57:53,33,10,7,,5Dg5-7AQTkDYXNf7hUXmaw
3,--0YW17u1XvJ75JTWzhzjw,Binnie,88,2011-01-14 20:21:51,88,18,19,"2012,2015","CJ6fkuVqqBojiQ7sOM7PKw, 4MMiTsh7PNtsClBeyZ5FUg..."
4,--1etymRjBb5FIFVv5d6Ig,Dina,2,2017-07-29 20:53:02,1,0,0,,None
5,--1orhUoGFSdHXsoxqQc8g,Keny,41,2011-03-18 05:56:39,18,2,10,,"eqN_zvrVaiWqu2SnxaALyg, YQjH1rzBHk308YTRRoDcFA..."
6,--1oZcRo9-QKOtTqREKB6g,Claire,37,2017-02-28 02:30:33,12,3,7,,None
7,--1ZsAsSdoqgoZZTYjsuiw,RW,17,2007-05-18 00:11:29,31,1,4,,None
8,--1_BaDq0xK5SPmFt1H1FQ,Nicole,5,2015-08-02 11:21:30,3,0,0,,"qI8Vv_A-2MU6yeSDGeACRQ, vq59HLmKyb13f2XKNW6V9Q..."
9,--2PnhMMH7EYoY3wywOvgQ,Andrew,28,2015-11-15 06:20:34,46,19,38,2018,"p85RMfRWOeRihfny0fqubA, GiAzFKBJ61RJp1WbApzgLg..."


In [9]:
# remove the useless space
user.replace('\s+','',regex=True,inplace=True)
user.head()

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends
0,---2PmXbF47D870stH1jqA,Susan,35,2012-10-2413:20:46,63,4,36,,"s7cUp9ma9h9FYN-fa0FtgA,o9QkuHIyxmqbORF7PeAbuw,..."
1,---7FS-3SMy-cnnIPHcW9w,Nick,1,2014-12-2816:02:17,0,0,0,,None
2,--0DrQkM0FT-yCQRWw82uQ,Jimmy,25,2007-11-1315:57:53,33,10,7,,5Dg5-7AQTkDYXNf7hUXmaw
3,--0YW17u1XvJ75JTWzhzjw,Binnie,88,2011-01-1420:21:51,88,18,19,"2012,2015","CJ6fkuVqqBojiQ7sOM7PKw,4MMiTsh7PNtsClBeyZ5FUg,..."
4,--1etymRjBb5FIFVv5d6Ig,Dina,2,2017-07-2920:53:02,1,0,0,,None


In [10]:
# save as a csv file
user.to_csv('./Users.csv',index=None)

In [ ]:
# after create a document collection, named 'Users', in Arangodb Oasis, then apply cmd to import user collection
arangoimport   --server.endpoint ssl://6bb0cac1a861.arangodb.cloud:8529 --server.username root 
--server.password=8nOX5cJqYt1uH4EYIupM 
--server.database yelpSNA --file “C:/Users/46241/Desktop/GWU/courses/Spring 2022/EMSE 6586/Jupyter_File/EMSE 6586/Project/Users.csv” 
--type csv --collection Users --translate "user_id =_key"

# 2. Create and import user-to-friends edge file

In [13]:
# split friends column
user = pd.read_sql("""select yelp.user.user_id as user_id, substring_index(substring_index(yelp.user.friends,',',b.help_topic_id+1),',',-1) as friends 
                        from yelp.user
                        join mysql.help_topic b on b.help_topic_id < (length(yelp.user.friends) - length(replace(yelp.user.friends,',',''))+1)
                        ;""",
                 db)
user.head(10)

,user_id,friends
0,---2PmXbF47D870stH1jqA,o9QkuHIyxmqbORF7PeAbuw
1,---2PmXbF47D870stH1jqA,s7cUp9ma9h9FYN-fa0FtgA
2,---2PmXbF47D870stH1jqA,WEXQ2ihDaMzGT_PGR7jJtw
3,---2PmXbF47D870stH1jqA,Q70zvjAa9aawgW7KsNVGDA
4,---2PmXbF47D870stH1jqA,amIU
5,---7FS-3SMy-cnnIPHcW9w,None
6,--0DrQkM0FT-yCQRWw82uQ,5Dg5-7AQTkDYXNf7hUXmaw
7,--0YW17u1XvJ75JTWzhzjw,4MMiTsh7PNtsClBeyZ5FUg
8,--0YW17u1XvJ75JTWzhzjw,CJ6fkuVqqBojiQ7sOM7PKw
9,--0YW17u1XvJ75JTWzhzjw,IcfmCuUmjBLFAsD2TxEsIQ


In [28]:
# remove users without friend
user_fri = user.replace('None',np.nan).dropna()

# show the basic information
print(user_fri.info())
display(user_fri.head(10))
nuser_num = user_fri['user_id'].nunique()
nfri_num = user_fri['friends'].nunique()
print(f"There are {nuser_num} unique user_id and {nfri_num} unique friend id in total.")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2021546 entries, 0 to 2261591
Data columns (total 2 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   user_id  object
 1   friends  object
dtypes: object(2)
memory usage: 46.3+ MB
None


,user_id,friends
0,---2PmXbF47D870stH1jqA,o9QkuHIyxmqbORF7PeAbuw
1,---2PmXbF47D870stH1jqA,s7cUp9ma9h9FYN-fa0FtgA
2,---2PmXbF47D870stH1jqA,WEXQ2ihDaMzGT_PGR7jJtw
3,---2PmXbF47D870stH1jqA,Q70zvjAa9aawgW7KsNVGDA
4,---2PmXbF47D870stH1jqA,amIU
6,--0DrQkM0FT-yCQRWw82uQ,5Dg5-7AQTkDYXNf7hUXmaw
7,--0YW17u1XvJ75JTWzhzjw,4MMiTsh7PNtsClBeyZ5FUg
8,--0YW17u1XvJ75JTWzhzjw,CJ6fkuVqqBojiQ7sOM7PKw
9,--0YW17u1XvJ75JTWzhzjw,IcfmCuUmjBLFAsD2TxEsIQ
10,--0YW17u1XvJ75JTWzhzjw,OlmMcMyP72X7Lx-BXQQSEA


There are 466420 unique user_id and 1587608 unique friend id in total.


In [16]:
# count the number of friend of each user
user_fri['user_id'].value_counts(ascending=False)

oxBKmHuS_L1k5BZWEZ0vZw    5
tNa28OqP3MuI4rwKylDqtQ    5
eaGgaAhuFVsSP0e5rEoqIg    5
AMvhhNcYvW73lf9PbAP8wA    5
6OdJvA5hkOU9JzVeO3py5Q    5
                         ..
-nfltJk17Du_wvQxKB84aw    1
i9nIfdMn4odcenhfKykhAQ    1
un_HobT53RLBoDNEnd5KOQ    1
awkxSRrt6qRSIjRTaJMIFA    1
R0WR5P4B9dbwMtGiGEUnDw    1
Name: user_id, Length: 466420, dtype: int64

In [26]:
# according to the naming rule of edge file,
# we add the vertex collection name in front of each id and change the column name to set 'from' and 'to' attributes to each node
user_fri["_from"] = 'Users/' + user["user_id"]
user_fri["_to"] = 'Users/' + user["friends"]
user_fri.drop(columns=['user_id', 'friends'],inplace=True)

# remove the useless space
user_fri.replace('\s+','',regex=True,inplace=True)


,_from,_to
0,Users/---2PmXbF47D870stH1jqA,Users/o9QkuHIyxmqbORF7PeAbuw
1,Users/---2PmXbF47D870stH1jqA,Users/s7cUp9ma9h9FYN-fa0FtgA
2,Users/---2PmXbF47D870stH1jqA,Users/WEXQ2ihDaMzGT_PGR7jJtw
3,Users/---2PmXbF47D870stH1jqA,Users/Q70zvjAa9aawgW7KsNVGDA
4,Users/---2PmXbF47D870stH1jqA,Users/amIU
6,Users/--0DrQkM0FT-yCQRWw82uQ,Users/5Dg5-7AQTkDYXNf7hUXmaw
7,Users/--0YW17u1XvJ75JTWzhzjw,Users/4MMiTsh7PNtsClBeyZ5FUg
8,Users/--0YW17u1XvJ75JTWzhzjw,Users/CJ6fkuVqqBojiQ7sOM7PKw
9,Users/--0YW17u1XvJ75JTWzhzjw,Users/IcfmCuUmjBLFAsD2TxEsIQ
10,Users/--0YW17u1XvJ75JTWzhzjw,Users/OlmMcMyP72X7Lx-BXQQSEA


In [27]:
# save as a csv file
user_fri.to_csv('./U&F.csv',index=None)

In [ ]:
# apply cmd to create an edge collection, named 'friends' in Arangodb Oasis and import edge data
arangoimport   --server.endpoint ssl://6bb0cac1a861.arangodb.cloud:8529 --server.username root --server.password=8nOX5cJqYt1uH4EYIupM 
--server.database yelpSNA --file “C:/Users/46241/Desktop/GWU/courses/Spring 2022/EMSE 6586/Jupyter_File/EMSE 6586/Project/U&F.csv” 
--type csv --collection friends --create-collection true --create-collection-type edge